## 14년

In [ ]:
small_years =  [14]

In [ ]:
import os
os.chdir('./../../')
from src.basic_utils import *
from src.modeling_utils import *
from src.pivot_utils import *

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data_path = 'data/'

train = pd.read_parquet(data_path+'train(song_trim).parquet')
val = pd.read_json(data_path+'val(lower).json')
# test = pd.read_json(data_path+'test(lower).json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
# tag_song_dict = load_json(data_path+'tag_song_dict.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
no_tag, no_song, yes_index, no_both = check_target_type(val)

In [ ]:
v1v3_index = no_tag + yes_index

train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )

year_list = list(train['year'].unique())
year_list= sorted(year_list)

v1v3 = val[val.index.isin(v1v3_index)]

val_tmp = v1v3[v1v3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)

In [ ]:
date_list = list(train['date'].unique())
date_list = sorted(date_list)

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = make_popular_date_dict('tags', train, date_list, 13)
popular_date_song = make_popular_date_dict('songs', train, date_list, 9)      

In [ ]:
tmp = train[(train['year'].isin(small_years))]
like_num = int(np.percentile(tmp['like_cnt'], 70))
tmp = tmp[(tmp['like_cnt'] > like_num)]
print("train best playlist", tmp.shape)
print('train+valid = ', tmp.shape[0]+val_tmp.shape[0])

In [ ]:
before_tmp = before_pivot(tmp)

## train만 1보다 작은 노래는 없애준다.
tmp_dict = before_tmp['song'].value_counts().to_dict()
before_tmp['count']= before_tmp['song'].apply(lambda x : tmp_dict[x])
before_tmp = before_tmp[before_tmp['count'] > 1]
print(before_tmp.shape)

before_val_tmp = before_pivot(val_tmp)

In [ ]:
final_result = pd.concat([before_tmp, before_val_tmp])
del before_tmp, before_val_tmp
gc.collect()

In [ ]:
def make_cosine_predict(final_result):
    R_df = final_result.pivot(index = 'id', columns ='song', values = 'point').fillna(0)
    del final_result
    
    print("make cosine data")    
    cosine_array = cosine_similarity(R_df, R_df)
    predicted_array = np.zeros(shape=(len(R_df.index),len(R_df.columns))) 
    for i in tqdm(range(len(cosine_array))):
        top_200 = cosine_array[i].argsort()[-201:][::-1]
        top_200 = np.delete(top_200, 0)

        weighted_sum = np.array([0])
        for top_idx in top_200:
            weighted_sum = weighted_sum + (cosine_array[i][top_idx] * R_df.values[top_idx])
        predicted = weighted_sum / len(top_200)
        predicted_array[i] = predicted
    iu_predicted = R_df.values*(-99999) + predicted_array
    
    print("make dic data")    
    cf_dic = {}
    for i in range(len(iu_predicted)):
        cf_dic[R_df.index[i]] = R_df.columns[iu_predicted[i].argsort()[-200:][::-1]].tolist()
    
    return cf_dic

In [ ]:
cf_dic = make_cosine_predict(final_result)

In [ ]:
val1_predict = []
for i in range(len(val_tmp)):
    ############### song 채우기 ##################################
    date = date_dict[val_tmp.iloc[i]['date']]
    row_number = val_tmp.iloc[i]['id']
    cur_song = list(cf_dic[row_number])
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in val_tmp.iloc[i]['songs']:
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    cur_tag = []
    for k in tag_counter[:10]:
        cur_tag.append(k[0])
    if (len(cur_tag) < 10) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_date_tag[date]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:10]
    if len(cur_tag) == 0:
        cur_tag = popular_date_tag[date]['tags'][:10]
        
    #############################################################    

    val1_predict.append({
        "id" : val_tmp.iloc[i]['id'],
        "songs": remove_seen(val_tmp.iloc[i]['songs'], cur_song)[:100],
        "tags": cur_tag,
    })
    

In [ ]:
## save file
write_json(val1_predict, 'test1+3_'+str(small_years)+'.json')